In [4]:
import pandas as pd
import numpy as np
import math
%matplotlib inline
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from lightgbm import LGBMRegressor
from sklearn import preprocessing
from sklearn import utils

In [5]:
#블랙록월드에너지증권자투자신탁(주식-재간접형)(H)A
# p1 = [728.2,733.27,734.23,735.07,751.21,749.21,747.86,752.64,756.37,759.72,753.08,749.73,748.04,727.3,718.71]

#DGB공모주플러스증권투자신탁1호(채권혼합)Ae
# p1 = [1093.49,1093.05,1092.44,1093.04,1093.18,1093.98,1091.9,1091.97,1090.37,1090.36,1089.33,1088.26,1085.32,1086.6,1087.06]

#DGB똑똑단기채증권투자신탁(채권)Ce
# p1 = [1027.38,1027.41,1027.44,1027.72,1027.77,1027.93,1027.99,1028.11,1028.18,1028.33,1028.43,1028.47,1028.51,1028.64,1028.68]

# 유진챔피언단기증권자투자신탁(채권)A
# p1 = [1035.13,1035.1,1035.04,1035.138,1035.43,1035.58,1035.63,1035.78,1035.75,1035.89,1036.01,1036.13,1036.24,1036.34,1036.38]

#이스트스프링차이나드래곤AShare증권자투자신탁(UH)(주식)C
# p1 = [787.02,796.32,795.68,815.41,808.35,811.7,815,819.85,827.03,827.18,792.22,788.92,762.85,769.36,760.32]

#미래에셋고배당포커스증권자투자신탁1호(주식)Ae
# p1 = [1046.69,1047.06,1046.49,1054.47,1056.71,1053.1,1047.23,1028.39,1021.61,1012.53,1012.27,1005.22,964.78,977.56,968.28]

#키움단기국공채증권자투자신탁제1호(채권)A
# p1 = [1024.67,1024.64,1024.52,1024.78,1024.73,1024.87,1024.91,1025.01,1024.97,1025.06,1025.15,1025.28,1025.35,1025.4,1025.4]

#트러스톤다이나믹코리아50증권자투자신탁(주식혼합)A
# p1 = [1024.67,1024.64,1024.52,1024.78,1024.73,1024.87,1024.91,1025.01,1024.97,1025.06,1025.15,1025.28,1025.35,1025.4,1025.4]

#AB미국그로스증권투자신탁(주식-재간접형)A
p1 = [1441.31,1449.98,1449.44,1460.47,1454.48,1460.65,1460.48,1461.1,1453.63,1434.05,1421.08,1412.36,1365.09,1351.41,1379.94]

#DGB똑똑가치배당증권자투자신탁(주식)Ae
# p1 = [913.32,919.65,904.8,945.99,956.23,964.45,965.89,980.49,991.6,995.06,997.49,987.26,983.77,978.84,978.91]

#KB밸류포커스30증권자투자신탁(채권혼합)A-E
# p1= [1039.79,1037.22,1035.06,1038.58,1041.35,1041.02,1040.06,1038.09,1033.15,1032.38,1028.56,1024.8,1014.55,1017.83,1015.61]

#NH-Amundi코리아2배레버리지증권투자신탁(주식-파생형)Ce
# p1 = [1092.29,1094.34,1113.27,1123.92,1140.76,1127.56,1124.2,1097.44,1061.15,1057.25,1048.21,1033.59,942.24,969.91,954.68]

#KB스타코리아리버스인덱스증권투자신탁(주식-파생형)C-E
# p1 = [852.6,852.1,845.41,841.96,836.59,840.45,841.86,850.57,863.65,865.4,868.33,873.72,909.85,895.84,902.57]




In [33]:
# data = pd.read_excel('블랙록월드에너지증권자투자신탁(주식-재간접형)(H)A.xlsx')
data = pd.read_excel('펀드123/미래에셋고배당포커스증권자투자신탁1호(주식)Ae.xlsx')
data = data.filter(regex=r'기준일|기준가')
data = data.sort_values(['기준일','기준가'])
melt2 = data.copy()

copi = pd.read_excel('코스피 지수.xls')
copi = copi.filter(regex=r'기준일|종가')
copi = copi.sort_values(['기준일','종가'])
data2 =copi.copy()
cosdac = pd.read_excel('코스닥지수.xls')
cosdac = cosdac.filter(regex=r'기준일|종가')
cosdac = cosdac.sort_values(['기준일','종가'])
data3 = cosdac.copy()

In [34]:
tr = data2[data2['기준일'] <= 115]
cx = tr[tr['기준일'] <= 100]
cy = tr[tr['기준일'] > 100]
cx = cx.drop(['기준일'], axis = 1)
cy = cy.drop(['기준일'], axis = 1)
cx = np.transpose(cx)
cy = np.transpose(cy)
for week in range(1,193,1): #203
    tr = copi[copi['기준일'] >= week+1]
    tr2 = tr[tr['기준일'] <= week+115]
    val = tr2[tr2['기준일'] > week+100]
    train = tr2[tr2['기준일'] <= week+100]

    xtr= train.drop(['기준일'], axis=1)
    ytr= val.drop(['기준일'], axis=1)
    xtr = np.transpose(xtr)
    ytr = np.transpose(ytr)
    cx = np.vstack((cx,xtr))
    cy = np.vstack((cy,ytr))

In [35]:
tr = data3[data3['기준일'] <= 115]
dx = tr[tr['기준일'] <= 100]
dy = tr[tr['기준일'] > 100]
dx = dx.drop(['기준일'], axis = 1)
dy = dy.drop(['기준일'], axis = 1)
dx = np.transpose(dx)
dy = np.transpose(dy)
for week in range(1,193,1):
    tr = copi[copi['기준일'] >= week+1]
    tr2 = tr[tr['기준일'] <= week+115]
    val = tr2[tr2['기준일'] > week+100]
    train = tr2[tr2['기준일'] <= week+100]

    xtr= train.drop(['기준일'], axis=1)
    ytr= val.drop(['기준일'], axis=1)
    xtr = np.transpose(xtr)
    ytr = np.transpose(ytr)
    dx = np.vstack((dx,xtr))
    dy = np.vstack((dy,ytr))
   # print(week)

In [36]:
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [37]:
mean_error = []
tr = melt2[melt2['기준일'] <= 115]
tx = tr[tr['기준일'] <= 100]
ty = tr[tr['기준일'] > 100]
tx = tx.drop(['기준일'], axis = 1)
ty = ty.drop(['기준일'], axis = 1)
tx = np.transpose(tx)
ty = np.transpose(ty)
for week in range(1,193,1):
    tr = melt2[melt2['기준일'] >= week+1]
    tr2 = tr[tr['기준일'] <= week+115]
    val = tr2[tr2['기준일'] > week+100]
    train = tr2[tr2['기준일'] <= week+100]

    xtr= train.drop(['기준일'], axis=1)
    ytr= val.drop(['기준일'], axis=1)
    xtr = np.transpose(xtr)
    ytr = np.transpose(ytr)
    tx = np.vstack((tx,xtr))
    ty = np.vstack((ty,ytr))
            
mdl = RandomForestRegressor(n_estimators=1000, criterion='mse', random_state=0, n_jobs=-1)
li = np.hstack((tx,cx))
li = np.hstack((li,dx))
mdl.fit(li, ty)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [38]:
test_error=[]
for step in range(193,205,1):  
    fund = melt2[melt2['기준일'] >= step]
    fund = fund[fund['기준일'] <= step+99]
    fund = fund.drop(['기준일'],axis = 1)
    fund = np.transpose(fund)
    fund2 = data2[data2['기준일'] >= step]
    fund2 = fund2[fund2['기준일'] <= step+99]
    fund2 = fund2.drop(['기준일'],axis = 1)
    fund2 = np.transpose(fund2)
    fund3 = data3[data3['기준일'] >= step]
    fund3 = fund3[fund3['기준일'] <= step+99]
    fund3 = fund3.drop(['기준일'],axis = 1)
    fund3 = np.transpose(fund3)
    fund4 = np.hstack((fund,fund2))
    fund4 = np.hstack((fund4,fund3))
    p = mdl.predict(fund4)
    print(p)
    i=0
    p = np.transpose(p)
# print(p[i])
# print(p1[i])
    
    last = step+114
    start = step+100
    for num in range(start,last,1):
        val = melt2[melt2['기준일'] == num]
        (val['기준가'].values)
        error = rmsle(val['기준가'].values, p[i])
        i=i+1
        print('Step %d - Error %.5f' % (num, error))
        mean_error.append(error)
    print('Mean Error = %.5f' % np.mean(mean_error))
    test_error.append(np.mean(mean_error))
print('Test Error = %.5f' % np.mean(test_error))


[[1019.37367 1020.59648 1018.5778  1018.84556 1017.27823 1021.51058
  1022.93797 1025.26275 1028.5739  1032.88762 1034.9109  1037.09272
  1039.16606 1043.26187 1043.92725]]
Step 293 - Error 0.00639
Step 294 - Error 0.00091
Step 295 - Error 0.00494
Step 296 - Error 0.00261
Step 297 - Error 0.00439
Step 298 - Error 0.00091
Step 299 - Error 0.00113
Step 300 - Error 0.00293
Step 301 - Error 0.00488
Step 302 - Error 0.00628
Step 303 - Error 0.00553
Step 304 - Error 0.00647
Step 305 - Error 0.00743
Step 306 - Error 0.01197
Mean Error = 0.00477
[[1022.59119 1022.4434  1021.6424  1021.19973 1020.13096 1020.72232
  1021.6525  1022.66924 1024.60016 1027.21027 1029.64954 1031.70531
  1033.35149 1036.55643 1038.62919]]
Step 294 - Error 0.00286
Step 295 - Error 0.00873
Step 296 - Error 0.00535
Step 297 - Error 0.00823
Step 298 - Error 0.00226
Step 299 - Error 0.00330
Step 300 - Error 0.00645
Step 301 - Error 0.01063
Step 302 - Error 0.01433
Step 303 - Error 0.01300
Step 304 - Error 0.01366
Step 305

In [426]:
test = melt2[melt2['기준일'] >=230]
i=0
p = np.transpose(p)
# print(p[i])
# print(p1[i])

for week in range(230,245):
#     val = melt2[melt2['기준일'] == week]
#     (val['기준가'].values)
    error = rmsle([p1[i]], p[i])
    i=i+1
    print('Week %d - Error %.5f' % (week, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

Week 230 - Error 0.00879
Week 231 - Error 0.01335
Week 232 - Error 0.01126
Week 233 - Error 0.01778
Week 234 - Error 0.01229
Week 235 - Error 0.01559
Week 236 - Error 0.01474
Week 237 - Error 0.01513
Week 238 - Error 0.01000
Week 239 - Error 0.00337
Week 240 - Error 0.01263
Week 241 - Error 0.01933
Week 242 - Error 0.05405
Week 243 - Error 0.06487
Week 244 - Error 0.04470
Mean Error = 0.02119


In [216]:
tp = np.ravel(p, order='C')

In [217]:
f = open('유진챔피언단기 예측결과.txt', mode='wt', encoding='utf-8')

In [218]:

for i in range(len(tp)):
    ts = str(round(tp[i],4))
    s = 'insert into fund10 values(null,'+ts+');\n'
    f.write(s)

f.close()